# Basic RAG

This notebook is a fully setup basic RAG app including

- PDF Loader
- Chunking
- Vector Embedding
- Pinecone Vector Database
- Retrieval
- Question/Response

## Pre-requisites

In [ ]:
# Create virtual environment
! python -m venv venv
! source venv/bin/activate
! which python #make sure that the end of your path output contains "...venv/bin/python"

In [ ]:
# Install all packages
! pip install -r requirements.txt --quiet

## Environment

`(1) Packages`

In [1]:
import os
from dotenv import load_dotenv

# Load all environment variables from .env file
load_dotenv()

# Access the environment variables
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_endpoint = os.getenv('LANGCHAIN_ENDPOINT')
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')

## LLM
openai_api_key = os.getenv('OPENAI_API_KEY')

## Pinecone Vector Database
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pinecone_api_host = os.getenv('PINECONE_API_HOST')
index_name = os.getenv('PINECONE_INDEX_NAME')


`(2) LangSmith`

https://docs.smith.langchain.com/

In [2]:
os.environ['LANGCHAIN_TRACING_V2'] = langchain_tracing_v2
os.environ['LANGCHAIN_ENDPOINT'] = langchain_endpoint
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key

`(3) API Keys`

In [3]:
os.environ['OPENAI_API_KEY'] = openai_api_key
openai_model = "gpt-3.5-turbo"

#Pinecone keys
os.environ['PINECONE_API_KEY'] = pinecone_api_key
os.environ['PINECONE_API_HOST'] = pinecone_api_host
os.environ['PINECONE_INDEX_NAME'] = index_name

`(4) Pinecone Init`

In [3]:
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
'''client = chromadb.PersistentClient(
    path="test",
    settings=Settings(),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)
'''
client = chromadb.HttpClient(
    host="192.168.107.42",
    port=8000,
    ssl=False,
    headers=None
)

index = client.get_or_create_collection("rag1")

In [ ]:
# Listar colecciones en la base de datos
collections = client.list_collections()
for collection in collections:
    print(collection.name) 

maria2
test
maria
rag1
test01
rag
maria_05


In [22]:

retriever.invoke("List active features in the configuration device n5k-2")


[Document(metadata={'_node_content': '{"id_": "28656bc6-6d45-4606-a2f3-cfe92c355977", "embedding": null, "metadata": {"file_path": "/root/ax-gen-ia/jupyter/../poc/data/n5k-2.cfg", "file_name": "n5k-2.cfg", "file_size": 35996, "creation_date": "2024-11-21", "last_modified_date": "2024-11-21"}, "excluded_embed_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "excluded_llm_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "relationships": {"1": {"node_id": "6e1fe364-6ba1-42a9-a7f3-65bb13072552", "node_type": "4", "metadata": {"file_path": "/root/ax-gen-ia/jupyter/../poc/data/n5k-2.cfg", "file_name": "n5k-2.cfg", "file_size": 35996, "creation_date": "2024-11-21", "last_modified_date": "2024-11-21"}, "hash": "46fd16f827d013c0074ba9d640236c7fc3e2b764ce1d9de73414b15ca93c6a2f", "class_name": "RelatedNodeInfo"}, "3": {"node_id": "8015477f-6aab-4f7c-aaf6-aaa213

In [19]:
print(results)

<coroutine object BaseRetriever.ainvoke at 0x7f21f9502260>


In [ ]:
result=index.get(
    ids=["id1"],
    include=["embeddings"]
)

print(result["embeddings"][0][:5])
print(len(result["embeddings"][0]))

AttributeError: 'Client' object has no attribute 'get'

In [ ]:
'''
from pinecone import Pinecone

pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])
index = pc.Index(os.environ['PINECONE_INDEX_NAME'])'''

## Full RAG App (Basic)

In [4]:
OLLAMA_HOST = '192.168.107.42'
OLLAMA_PORT = '11434'
OLLAMA_URL = f"http://{OLLAMA_HOST}:{OLLAMA_PORT}"

OLLAMA_MODEL= 'llama3.1'
OLLAMA_NUM_THREADS= 8
OLLAMA_TEMP=0
HUGGINGFACE_EMBEDDING = 'sentence-transformers/all-mpnet-base-v2'

In [16]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
#from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_ollama.llms import OllamaLLM
from langchain_ollama.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate

#### INDEXING ####

pdf_file_paths = "test/scikit-dataset-transformations-feature-extraction.pdf"
loader = PyPDFLoader(pdf_file_paths)

docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Embed
'''vectorstore = Chroma.from_documents(client=client,
    documents=splits, 
    embedding=embeddings,
    collection_name='test01'
)'''

# Usar Chroma como almacén vectorial
vectorstore = Chroma(
    client=client,
    collection_name="rag1",
    embedding_function=embeddings
    

)

retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# LLM
llm = ChatOllama(
  base_url=OLLAMA_URL,
  model=OLLAMA_MODEL,
  num_thread=OLLAMA_NUM_THREADS,
  temperature=OLLAMA_TEMP
)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# Question
print(rag_chain.invoke("List active features in the configuration device mq1-n5k-1"))

Based on the provided configuration, here are the active features:

1. **VPC (Virtual Port Channel)**:
	* `vpc 4` on interface port-channel5
	* `vpc 5` on interface port-channel5
	* `vpc 15` on interface port-channel15
	* `vpc 100` on interface port-channel100
2. **FEX (Fabric Extender)**:
	* `fex associate 101` on interface port-channel15
3. **VSAN (Virtual Storage Area Network)**:
	* `vsan database` with multiple VSANs (10) and interfaces associated with them
4. **Port Channels**:
	* Port channel5 is active, with a description of "mq1-ucshyfx-b"
	* Port channel10 is active, with a description of "mq-fas2552-a - Uplink Ethernet"
	* Port channel15 is active, with a description of "Uplink Corporativo - mq1-mgsw-1"
	* Port channel100 is active, with a description of "Uplink Corporativo - mq1-mgsw-1"
5. **Trunking**:
	* Trunking is enabled on multiple interfaces (e.g., Ethernet1/1, Ethernet1/2, port-channel5)
6. **VFC (Virtual Fabric Channel)**:
	* Multiple VFCs are bound to physical inte

In [ ]:
# Usar el retriever para recuperar el contexto
retriever.get_relevant_documents(query='List active features in the configuration device mq1-n5k-2')

[Document(metadata={'_node_content': '{"id_": "28656bc6-6d45-4606-a2f3-cfe92c355977", "embedding": null, "metadata": {"file_path": "/root/ax-gen-ia/jupyter/../poc/data/n5k-2.cfg", "file_name": "n5k-2.cfg", "file_size": 35996, "creation_date": "2024-11-21", "last_modified_date": "2024-11-21"}, "excluded_embed_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "excluded_llm_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "relationships": {"1": {"node_id": "6e1fe364-6ba1-42a9-a7f3-65bb13072552", "node_type": "4", "metadata": {"file_path": "/root/ax-gen-ia/jupyter/../poc/data/n5k-2.cfg", "file_name": "n5k-2.cfg", "file_size": 35996, "creation_date": "2024-11-21", "last_modified_date": "2024-11-21"}, "hash": "46fd16f827d013c0074ba9d640236c7fc3e2b764ce1d9de73414b15ca93c6a2f", "class_name": "RelatedNodeInfo"}, "3": {"node_id": "8015477f-6aab-4f7c-aaf6-aaa213

In [5]:
print(rag_chain.invoke("Show me banner motd in mq1-n5k-1"))

***********************************
**     Acuntia Maqueta Sistemas  **
**            mq1-n5k-1          **
***********************************


In [24]:
print(rag_chain.invoke("List active features in the configuration device n5k-2"))

There is no "n5k-2" in the provided output, but I assume you meant to ask about the device "nx-osv9000-4".

Here are the active features listed in the configuration:

1. **Telnet**: `feature telnet`
2. **OSPF**: `feature ospf`
3. **BGP**: `feature bgp`
4. **PIM**: `feature pim`
5. **Fabric Forwarding**: `feature fabric forwarding`
6. **Interface VLAN**: `feature interface-vlan`
7. **VN-SEGMENT-VLAN-BASED**: `feature vn-segment-vlan-based`

These are the features that are currently enabled on the device "nx-osv9000-4".


In [25]:
print(rag_chain.invoke("Show me banner motd in n5k-2"))

To show the banner motd on n5k-2, you can use the following command:

```
show banner motd
```

This will display the message of the day (motd) that is currently configured on the device. 

In your case, since you provided a configuration snippet for n5k-1, I'll assume that the output would be similar for n5k-2 as well.

Here's what it might look like:

```
***********************************
**     Acuntia Maqueta Sistemas  **
**            mq1-n5k-1          **
***********************************
```


In [26]:
print(rag_chain.invoke("List active features in the configuration device mq1-n5k-2"))

Based on the provided output, I can see that the hostname of the device is "nx-osv9000-4", not "mq1-n5k-2". However, I will provide the list of active features for the given device.

Here are the active features:

* telnet
* ospf
* bgp
* pim
* fabric forwarding
* interface-vlan
* vn-segment-vlan-based
* nv overlay

These features are enabled on the device "nx-osv9000-4".


In [23]:
print(rag_chain.invoke("tienes almacenada información sobre la configuracion de mq1-n5k-2?"))

Sí, tengo acceso a la configuración de `mq1-n5k-2` que se proporciona en el texto. Sin embargo, es importante destacar que esta configuración parece ser específica para un entorno de laboratorio o prueba y puede no reflejar una configuración realista o segura para un entorno de producción.

La configuración incluye:

* Configuración de SNMP
* Creación de interfaces de red (port-channel, vfc, fc)
* Asociación de interfaces a VSAN 10
* Configuración de velocidad y modo de operación para algunas interfaces

Si necesitas ayuda específica con alguna parte de esta configuración o si tienes preguntas sobre cómo funciona algo en particular, estaré encantado de ayudarte.


In [12]:
print(rag_chain.invoke("tienes almacenada información sobre la configuracion de mq1-n5k-2?"))

Sí, tengo acceso a la configuración de `mq1-n5k-2` que se proporciona en el texto. Sin embargo, es importante destacar que esta configuración parece ser específica para un entorno de laboratorio o prueba y puede no reflejar una configuración realista o segura para un entorno de producción.

La configuración incluye:

* Configuración de SNMP
* Creación de interfaces de red (port-channel, vfc, fc)
* Asociación de interfaces a VSAN 10
* Configuración de velocidad y modo de operación para algunas interfaces

Si necesitas ayuda específica con alguna parte de esta configuración o si tienes preguntas sobre cómo funciona algo en particular, estaré encantado de ayudarte.


In [14]:
print(rag_chain.invoke("me puedes proporcionar el banner de la configuracion de  mq1-n5k-2?"))

Lo siento, pero no puedo acceder a información específica de un dispositivo o sistema en particular. Sin embargo, puedo ayudarte a analizar y comprender la configuración que has proporcionado.

La configuración parece ser para un switch Cisco Nexus 5000 (N5K) con varias interfaces y configuraciones relacionadas con VPC (Virtual Port Channel), FEX (Fabric Extender), VSAN (Virtual Storage Area Network), y otros aspectos de red. 

Aquí hay algunos puntos destacados que podrían ser relevantes:

- **VPC Configuración**: Se configuran varios VPCs, lo que permite a los dispositivos conectarse como si estuvieran en el mismo switch físico, aunque puedan estar en diferentes switches o incluso en diferentes ubicaciones geográficas.

- **FEX Configuración**: La configuración de FEX (Fabric Extender) permite conectar dispositivos externos al switch principal como si fueran interfaces del propio switch. Esto puede ser útil para extender la red y agregar más dispositivos sin necesidad de cables adici